In [1]:
import pandas as pd
from pandas import DataFrame
import sqlite3

In [14]:
conn = sqlite3.connect('./data/checking-logs.sqlite')
#cursor = conn.cursor()

In [3]:
datamart   = pd.io.sql.read_sql(' SELECT c.uid, c.labname, c.timestamp AS first_commit_ts, MIN (p.datetime) as first_view_ts\
                                  FROM checker as c \
                                  LEFT JOIN pageviews as p\
                                  ON c.uid = p.uid\
                                  WHERE c.status = "ready" \
                                    AND c.numTrials = 1 \
                                    AND c.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1")\
                                    AND c.uid LIKE "user_%" \
                                  GROUP by c.timestamp, c.uid ',
                                conn, parse_dates = ['first_commit_ts', 'first_view_ts'])
datamart.to_sql('datamart', conn, if_exists = 'replace')

140

In [4]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [5]:
test = datamart.dropna()
test.to_sql('test', conn, if_exists='replace')
test.head(10)

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [15]:
control = datamart['first_view_ts'].isnull()
control.loc[:, 'first_view_ts'] = test.loc[:, 'first_view_ts'].mean()
control.to_sql('control', conn, if_exists='replace')
control.head(10)

TypeError: Could not convert 2020-04-18 10:56:55.8338992020-04-17 22:46:26.7850352020-04-17 22:46:26.7850352020-04-18 10:53:52.6234472020-04-18 10:53:52.6234472020-04-21 20:30:38.0349662020-05-09 23:54:54.2607912020-04-22 22:40:36.8240812020-04-17 22:46:26.7850352020-04-22 22:40:36.8240812020-05-10 21:07:50.3509462020-04-18 10:56:55.8338992020-05-08 10:53:47.1238322020-05-08 10:53:47.1238322020-04-18 12:19:50.1827142020-04-18 12:19:50.1827142020-04-18 10:56:55.8338992020-05-10 21:07:50.3509462020-04-26 21:53:59.6241362020-04-26 21:53:59.6241362020-04-26 22:49:29.2432782020-04-26 22:49:29.2432782020-05-09 23:54:54.2607912020-04-18 10:53:52.6234472020-05-08 10:53:47.1238322020-05-09 23:54:54.2607912020-05-10 21:07:50.3509462020-04-18 12:19:50.1827142020-04-17 22:46:26.7850352020-04-18 10:56:55.8338992020-04-22 22:40:36.8240812020-04-21 20:30:38.0349662020-04-26 21:53:59.6241362020-04-26 22:49:29.2432782020-04-18 10:53:52.6234472020-05-08 10:53:47.1238322020-05-09 23:54:54.2607912020-04-2

In [7]:
#control.first_view_ts = test.first_view_ts.mean()


In [8]:
test = pd.io.sql.read_sql('SELECT * FROM test', conn)

In [9]:
control = pd.io.sql.read_sql('SELECT * FROM control', conn)

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            59 non-null     int64 
 1   uid              59 non-null     object
 2   labname          59 non-null     object
 3   first_commit_ts  59 non-null     object
 4   first_view_ts    59 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.4+ KB


In [11]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            81 non-null     int64 
 1   uid              81 non-null     object
 2   labname          81 non-null     object
 3   first_commit_ts  81 non-null     object
 4   first_view_ts    81 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.3+ KB


In [12]:
conn.close()